In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-large-uncased-vocab.txt
/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased-vocab.txt
/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-large-uncased/pytorch_model.bin
/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-large-uncased/bert_config.json
/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased/pytorch_model.bin
/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/sacremoses/sacremoses-master/CONTRIBUTORS.md
/kaggle/input/sacremoses/sacremoses-master/README.md
/kaggle/input/sacremoses/sacremoses-master/.appveyor.yml
/kaggle/input/sacremoses/sacremoses-master/requireme

In [2]:
!pip install ../input/sacremoses/sacremoses-master/
!pip install ../input/transformers/transformers-master/

Processing /kaggle/input/sacremoses/sacremoses-master
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=882724 sha256=b538eaef3e5e93f28b06b834c1d7dc96c0687b0f9070300472e3e0c8f7a96d33
  Stored in directory: /root/.cache/pip/wheels/82/48/4b/05cb49d913a40c9d76f97931cd747d72fb17a77b0f6415cdba
Successfully built sacremoses
Processing /kaggle/input/transformers/transformers-master
  Created wheel for transformers: filename=transformers-2.2.0-cp36-none-any.whl size=362783 sha256=aeb120ab7b7eb1863d4ef2386f455872c4a82bf1ae83a7144f4093c487145109
  Stored in directory: /root/.cache/pip/wheels/ce/f3/1a/ee7248890cb4b8e8975988b1a67999e2d09ef54ce8ee815255
Successfully built transformers


In [3]:
import gc
import re
import os
import sys
import time
import pickle
import random
import unidecode
from tqdm import tqdm
tqdm.pandas()
from scipy.stats import spearmanr
from gensim.models import Word2Vec
from flashtext import KeywordProcessor
from keras.preprocessing import text, sequence

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder,MinMaxScaler


Using TensorFlow backend.


In [4]:
train = pd.read_csv('/kaggle/input/google-quest-challenge/train.csv')
test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')
sub = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')


In [5]:
import gc
import re
import os
import sys
import time
import pickle
import random

from tqdm import tqdm
from scipy.stats import spearmanr
from gensim.models import Word2Vec
from flashtext import KeywordProcessor
from keras.preprocessing import text, sequence

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from math import floor, ceil
from transformers import AdamW
from transformers.configuration_albert import AlbertConfig
from transformers.modeling_bert import ACT2FN, BertEmbeddings, BertSelfAttention, prune_linear_layer
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from transformers.optimization import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DistilBertModel, DistilBertTokenizer, DistilBertConfig


In [6]:
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder


In [7]:
PUNCTS = {
            '》', '〞', '¢', '‹', '╦', '║', '♪', 'Ø', '╩', '\\', '★', '＋', 'ï', '<', '?', '％', '+', '„', 'α', '*', '〰', '｟', '¹', '●', '〗', ']', '▾', '■', '〙', '↓', '´', '【', 'ᴵ',
            '"', '）', '｀', '│', '¤', '²', '‡', '¿', '–', '」', '╔', '〾', '%', '¾', '←', '〔', '＿', '’', '-', ':', '‧', '｛', 'β', '（', '─', 'à', 'â', '､', '•', '；', '☆', '／', 'π',
            'é', '╗', '＾', '▪', ',', '►', '/', '〚', '¶', '♦', '™', '}', '″', '＂', '『', '▬', '±', '«', '“', '÷', '×', '^', '!', '╣', '▲', '・', '░', '′', '〝', '‛', '√', ';', '】', '▼',
            '.', '~', '`', '。', 'ə', '］', '，', '{', '～', '！', '†', '‘', '﹏', '═', '｣', '〕', '〜', '＼', '▒', '＄', '♥', '〛', '≤', '∞', '_', '[', '＆', '→', '»', '－', '＝', '§', '⋅', 
            '▓', '&', 'Â', '＞', '〃', '|', '¦', '—', '╚', '〖', '―', '¸', '³', '®', '｠', '¨', '‟', '＊', '£', '#', 'Ã', "'", '▀', '·', '？', '、', '█', '”', '＃', '⊕', '=', '〟', '½', '』',
            '［', '$', ')', 'θ', '@', '›', '＠', '｝', '¬', '…', '¼', '：', '¥', '❤', '€', '−', '＜', '(', '〘', '▄', '＇', '>', '₤', '₹', '∅', 'è', '〿', '「', '©', '｢', '∙', '°', '｜', '¡', 
            '↑', 'º', '¯', '♫', '#'
          }


mispell_dict = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not",
"couldnt" : "could not", "didn't" : "did not", "doesn't" : "does not",
"doesnt" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
"haven't" : "have not", "havent" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "I would",
"i'd" : "I had", "i'll" : "I will", "i'm" : "I am", "isn't" : "is not", "it's" : "it is",
"it'll":"it will", "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", 
"shan't" : "shall not", "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "shouldnt" : "should not",
"that's" : "that is", "thats" : "that is", "there's" : "there is", "theres" : "there is", "they'd" : "they would", "they'll" : "they will",
"they're" : "they are", "theyre":  "they are", "they've" : "they have", "we'd" : "we would", "we're" : "we are", "weren't" : "were not",
"we've" : "we have", "what'll" : "what will", "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is",
"who'd" : "who would", "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not", "wouldn't" : "would not", "you'd" : "you would",
"you'll" : "you will", "you're" : "you are", "you've" : "you have", "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not", "tryin'":"trying"}


def clean_punct(text):
    text = str(text)
    for punct in PUNCTS:
        text = text.replace(punct, ' {} '.format(punct))
    return text

In [8]:
kp = KeywordProcessor(case_sensitive=True)
for k, v in mispell_dict.items():
    kp.add_keyword(k, v)
    

In [9]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'(\&lt)|(\&gt)', ' ', text)
    
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', text)
    text = kp.replace_keywords(text)
    text = clean_punct(text)
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [10]:
train['clean_title'] = train['question_title'].apply(lambda x : preprocessing(x))
train['clean_body'] = train['question_body'].apply(lambda x : preprocessing(x))
train['clean_answer'] = train['answer'].apply(lambda x : preprocessing(x))

test['clean_title'] = test['question_title'].apply(lambda x : preprocessing(x))
test['clean_body'] = test['question_body'].apply(lambda x : preprocessing(x))
test['clean_answer'] = test['answer'].apply(lambda x : preprocessing(x))

In [11]:
y_columns = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [12]:
find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])
ohe = OneHotEncoder()
ohe.fit(merged)

features_train = ohe.transform(train[features]).toarray()
features_test = ohe.transform(test[features]).toarray()

In [13]:
tokenizer = BertTokenizer.from_pretrained("/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased-vocab.txt")


In [14]:
def trim_input(tokenizer, title, body):
    all_title = []
    all_body = []
    all_answer = []
    for t, b in tqdm(zip(title, body), total=len(title)):
        
        tokenizer_t = tokenizer.tokenize(t)
        tokenizer_b = tokenizer.tokenize(b)
        
        t_len = len(tokenizer_t)
        b_len = len(tokenizer_b)
        #a_len = len(tokenizer_a)
                
        if b_len > BODY_MAX_LEN:
            tokenizer_b = tokenizer_b[:BODY_MAX_LEN]
            
                
        if t_len > TITLE_MAX_LEN:
            tokenizer_t = tokenizer_t[:TITLE_MAX_LEN]
        
        all_title.append(tokenizer_t)
        all_body.append(tokenizer_b)
        
    return all_title, all_body

def trim_input_a(tokenizer, answer):

    all_answer = []
    for a in tqdm(answer, total=len(answer)):

        tokenizer_a = tokenizer.tokenize(a)

        a_len = len(tokenizer_a) 
        if a_len > ANSWER_MAX_LEN:
            tokenizer_a = tokenizer_a[:ANSWER_MAX_LEN]

        all_answer.append(tokenizer_a)
        
    return all_answer

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = get_masks(stoken, max_sequence_length)
    input_segments = get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def convert_lines(tokenizer, title, body, max_seq_length=350):
    title, body = trim_input(tokenizer, title, body)
    all_tokens = []
    longer = 0
    
    all_tokens = []
    input_ids, input_masks, input_segments = [], [], []
    for i, (t, b) in tqdm(enumerate(zip(title, body)), total=len(title)):
        stoken = ["[CLS]"] + t + ["[SEP]"] + b + ["[SEP]"]
        
        ids = get_ids(stoken, tokenizer, max_seq_length)
        masks = get_masks(stoken, max_seq_length)
        segments = get_segments(stoken, max_seq_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]


def convert_lines_a(tokenizer, answer, max_seq_length=350):
    answer = trim_input_a(tokenizer, answer)
    all_tokens = []
    longer = 0
    
    all_tokens = []
    input_ids, input_masks, input_segments = [], [], []
    for i, a in tqdm(enumerate(answer), total=len(answer)):
        stoken = ["[CLS]"] + a + ["[SEP]"]
        
        ids = get_ids(stoken, tokenizer, max_seq_length)
        masks = get_masks(stoken, max_seq_length)
        segments = get_segments(stoken, max_seq_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

# 29 254
TITLE_MAX_LEN = 50
BODY_MAX_LEN = 297
ANSWER_MAX_LEN = 348

x_train_q = convert_lines(tokenizer, train['clean_title'], train['clean_body'])
x_test_q = convert_lines(tokenizer, test['clean_title'], test['clean_body'])

x_train_a = convert_lines_a(tokenizer, train['clean_answer'])
x_test_a = convert_lines_a(tokenizer, test['clean_answer'])




100%|██████████| 476/476 [00:00<00:00, 3144.31it/s]


In [15]:
def add_question_metadata_features(text):
    indirect = 0
    question_count = 0
    reason_explanation_words = 0
    choice_words = 0

    if '?' in text and '?' == text[-1]: 
        question_count += 1
        for token in text.split():
            if token.lower() == 'why':  # question_type_reason_explanation e.g index->102
                reason_explanation_words += 1
            elif token.lower() == 'or':
                choice_words += 1  

    if question_count == 0:
        indirect = 1
    return [indirect, question_count, reason_explanation_words, choice_words]

ans_user_category = train[train[['answer_user_name', 'category']].duplicated()][['answer_user_name', 'category']].values.tolist()
print(len(ans_user_category))


def question_answer_author_same(df):
    
    q_username = df['question_user_name']
    a_username = df['answer_user_name'] 
    
    author_same = []
    for i in range(len(df)):
        if q_username[i] == a_username[i]:
            author_same.append(int(1))
        else:
            author_same.append(int(0))
            
    return author_same

def count_mark(text):
    count = 0
    for char in text:
        if '?' == char:
            count += 1
      
    return count

def add_external_features(df):
    
    df['clean_title']      = df['clean_title'].apply(lambda x:str(x))
    df['title_num_words'] = df.clean_title.str.count('\S+')

    #If the question is longer, it may be more clear, which may help users give a more 
    df['clean_body']      = df['clean_body'].apply(lambda x:str(x))
    df['question_num_words'] = df.clean_body.str.count('\S+')
    
    #The assumption here is that longer answer could bring more useful detail
    df['clean_answer']            = df['clean_answer'].apply(lambda x:str(x))
    df['answer_num_words']  = df.clean_answer.str.count('\S+')
    
    #if the question is long and the answer is short, it may be less relevant
    df["question_vs_answer_length"] = df['question_num_words'] /  df['answer_num_words']

    df["total_text_length"] = df['title_num_words'] + df['question_num_words'] + df['answer_num_words']
    
    #if answer's author is the same as the corresponding question's author,
    #Why he/she asked question.. :)
    df["q_a_author_same"] = question_answer_author_same(df)
    
    df['title_token_len'] = df['clean_title'].apply(lambda x:len(x.split()))
    df['body_token_len'] = df['clean_body'].apply(lambda x:len(x.split()))
    df['answer_token_len'] = df['clean_answer'].apply(lambda x:len(x.split()))
    

    df['title_question_mark_count'] = df['clean_title'].apply(lambda x: count_mark(x))
    df['body_question_mark_count'] = df['clean_body'].apply(lambda x: count_mark(x))

    #answers which was posted by users who answer one category more than one times, they may have read more similar questions.
    #thus, the answers by this type of user will more relevent to question.
    ans_user_cat = []
    for x in tqdm(df[['answer_user_name', 'category']].values.tolist()):
        if x in ans_user_category:
            ans_user_cat.append(int(1))
        else:
            ans_user_cat.append(int(0))
    df['ans_user_with_cat'] = ans_user_cat
    
    handmade_features = []

    for idx, text in enumerate(df['clean_body'].values):
        handmade_features.append(add_question_metadata_features(text))
        

    return df, np.array(handmade_features)


train, train_handmade_features = add_external_features(train)
test, test_handmade_features   = add_external_features(test)

for c in ['title_num_words', 'question_num_words', 'answer_num_words', 'total_text_length', 'title_token_len', 'body_token_len', 'answer_token_len', 'title_question_mark_count', 'body_question_mark_count']:
    minmax = MinMaxScaler()
    temp_df = pd.concat([train[[c]], test[[c]]])
    minmax.fit(temp_df[c].values.reshape(-1, 1))
    train[c]= minmax.transform(train[c].values.reshape(-1, 1))
    test[c]= minmax.transform(test[c].values.reshape(-1, 1))

additonal_with_content = [
   'title_num_words', 'question_num_words', 'answer_num_words',
   'question_vs_answer_length', 'total_text_length', 'q_a_author_same',
   'title_token_len', 'body_token_len', 'answer_token_len',
   'ans_user_with_cat', 'title_question_mark_count',
   'body_question_mark_count']


       

1752


100%|██████████| 476/476 [00:00<00:00, 14843.78it/s]


In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [17]:
class TextDataset(torch.utils.data.TensorDataset):

    def __init__(self, x_train_q, x_train_a,  net_loc, additonal_content_feature, idxs, targets=None):
        self.input_ids = x_train_q[0][idxs]
        self.input_masks = x_train_q[1][idxs]
        self.input_segments = x_train_q[2][idxs]

        self.input_ids_a = x_train_a[0][idxs]
        self.input_masks_a = x_train_a[1][idxs]
        self.input_segments_a = x_train_a[2][idxs]
        self.net_loc = net_loc[idxs]
        self.additonal_content_feature = additonal_content_feature[idxs]
        self.targets = targets[idxs] if targets is not None else np.zeros((x_train_q[0].shape[0], 30))

    def __getitem__(self, idx):
        input_ids =  self.input_ids[idx]
        input_masks = self.input_masks[idx]
        input_segments = self.input_segments[idx]

        input_ids_a =  self.input_ids_a[idx]
        input_masks_a = self.input_masks_a[idx]
        input_segments_a = self.input_segments_a[idx]
        net_loc = self.net_loc[idx]
        target = self.targets[idx]
        additonal_content_feature = self.additonal_content_feature[idx]
        
        return input_ids, input_masks, input_segments, input_ids_a, input_masks_a, input_segments_a, net_loc, additonal_content_feature, target

    def __len__(self):
        return len(self.input_ids)

In [18]:
accumulation_steps = 1

SEED = 1312
NFOLDS = 5
BATCH_SIZE = 4
EPOCHS = 2
LR = 3e-5
seed_everything(SEED)

In [19]:
import math
def gelu(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(math.pi / 2) * (x + 0.044715 * x ** 3)))


In [20]:
class BertForSequenceClassification(torch.nn.Module):


    def __init__(self, additional=False):
        super(BertForSequenceClassification, self).__init__()
        
        bert_model_config = '/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
        
        bert_config = BertConfig.from_json_file(bert_model_config)
        bert_config_a = BertConfig.from_json_file(bert_model_config)
        bert_config.num_labels = 21
        bert_config_a.num_labels = 9
        bert_config.output_hidden_states=True
        bert_config_a.output_hidden_states=True
        
        self.albert = BertModel.from_pretrained('/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased/', config=bert_config)
        self.albert2 = BertModel.from_pretrained('/kaggle/input/bert-pretrained-model/pretrained-bert-models-for-pytorch/bert-base-uncased/', config=bert_config_a)

        self.dropout = nn.Dropout(0.1)
        #768 1024
        self.additional = additional
        if self.additional:
            self.additional_net_loc_linear = nn.Linear(64, 32)
            self.classifier_q = nn.Linear(768*2+32, 21)
            self.classifier_a = nn.Linear(768*2+32, 9)
        else:
            self.classifier_q = nn.Linear(768*2, 21)
            self.classifier_a = nn.Linear(768*2, 9)
            
#         self.init_weights()
        torch.nn.init.xavier_normal_(self.classifier_q.weight)
        torch.nn.init.xavier_normal_(self.classifier_a.weight)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        input_ids_a=None,
        attention_mask_a=None,
        token_type_ids_a=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        additional_feature=None,
        additional_feature_content=None,
    ):
        


        outputs_q = self.albert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        outputs_a = self.albert2(
            input_ids=input_ids_a,
            attention_mask=attention_mask_a,
            token_type_ids=token_type_ids_a,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
            
        hidden_states_q = outputs_q[-1]
        hidden_states_a = outputs_a[-1]
        
        h12_q = hidden_states_q[-1][:, 0].reshape((-1, 1, 768))
        h11_q = hidden_states_q[-2][:, 0].reshape((-1, 1, 768))

        h12_a = hidden_states_a[-1][:, 0].reshape((-1, 1, 768))
        h11_a = hidden_states_a[-2][:, 0].reshape((-1, 1, 768))

        all_h_q = torch.cat([h11_q, h12_q], 1)
        mean_pool_q = torch.mean(all_h_q, 1)
        max_pool_q, _ = torch.max(all_h_q, 1)

        all_h_a = torch.cat([h11_a, h12_a], 1)
        mean_pool_a = torch.mean(all_h_a, 1)
        max_pool_a, _ = torch.max(all_h_a, 1)

        pooled_output_q = torch.cat((mean_pool_q, max_pool_q), 1)
        pooled_output_a = torch.cat((mean_pool_a, max_pool_a), 1)
        
        pooled_output_q = self.dropout(pooled_output_q)
        pooled_output_a = self.dropout(pooled_output_a)


        logits_q = self.classifier_q(pooled_output_q)
        logits_a = self.classifier_a(pooled_output_a)

        return logits_q, logits_a  # (loss), logits, (hidden_states), (attentions)


In [21]:

# Author: Trent J. Bradberry <trentjason@hotmail.com>
# License: BSD 3 clause

import numpy as np

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, \
    BaseShuffleSplit, _validate_shuffle_split


def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds


class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits, shuffle, random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


class RepeatedMultilabelStratifiedKFold(_RepeatedSplits):
    """Repeated Multilabel Stratified K-Fold cross validator.
    Repeats Mulilabel Stratified K-Fold n times with different randomization
    in each repetition.
    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.
    n_repeats : int, default=10
        Number of times cross-validator needs to be repeated.
    random_state : None, int or RandomState, default=None
        Random state to be used to generate random state for each
        repetition as well as randomly breaking ties within the iterative
        stratification algorithm.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2, n_repeats=2,
    ...     random_state=0)
    >>> for train_index, test_index in rmskf.split(X, y):
    ...     print("TRAIN:", train_index, "TEST:", test_index)
    ...     X_train, X_test = X[train_index], X[test_index]
    ...     y_train, y_test = y[train_index], y[test_index]
    ...
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [0 1 4 5] TEST: [2 3 6 7]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    See also
    --------
    RepeatedStratifiedKFold: Repeats (Non-multilabel) Stratified K-Fold
    n times.
    """
    def __init__(self, n_splits=5, n_repeats=10, random_state=None):
        super(RepeatedMultilabelStratifiedKFold, self).__init__(
            MultilabelStratifiedKFold, n_repeats, random_state,
            n_splits=n_splits)


class MultilabelStratifiedShuffleSplit(BaseShuffleSplit):
    """Multilabel Stratified ShuffleSplit cross-validator
    Provides train/test indices to split data into train/test sets.
    This cross-validation object is a merge of MultilabelStratifiedKFold and
    ShuffleSplit, which returns stratified randomized folds for multilabel
    data. The folds are made by preserving the percentage of each label.
    Note: like the ShuffleSplit strategy, multilabel stratified random splits
    do not guarantee that all folds will be different, although this is
    still very likely for sizeable datasets.
    Parameters
    ----------
    n_splits : int, default 10
        Number of re-shuffling & splitting iterations.
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set to 0.1.
        The default will change in version 0.21. It will remain 0.1 only
        if ``train_size`` is unspecified, otherwise it will complement
        the specified ``train_size``.
    train_size : float, int, or None, default is None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedShuffleSplit that only uses
        random_state when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5,
    ...    random_state=0)
    >>> msss.get_n_splits(X, y)
    3
    >>> print(mss)       # doctest: +ELLIPSIS
    MultilabelStratifiedShuffleSplit(n_splits=3, random_state=0, test_size=0.5,
                                     train_size=None)
    >>> for train_index, test_index in msss.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    TRAIN: [1 2 5 6] TEST: [0 3 4 7]
    Notes
    -----
    Train and test sizes may be slightly different from desired due to the
    preference of stratification over perfectly sized folds.
    """

    def __init__(self, n_splits=10, test_size="default", train_size=None,
                 random_state=None):
        super(MultilabelStratifiedShuffleSplit, self).__init__(
            n_splits, test_size, train_size, random_state)

    def _iter_indices(self, X, y, groups=None):
        n_samples = _num_samples(X)
        y = check_array(y, ensure_2d=False, dtype=None)
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(
                    type_of_target_y))

        n_train, n_test = _validate_shuffle_split(n_samples, self.test_size,
                                                  self.train_size)

        n_samples = y.shape[0]
        rng = check_random_state(self.random_state)
        y_orig = y.copy()

        r = np.array([n_train, n_test]) / (n_train + n_test)

        for _ in range(self.n_splits):
            indices = np.arange(n_samples)
            rng.shuffle(indices)
            y = y_orig[indices]

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            test_idx = test_folds[np.argsort(indices)] == 1
            test = np.where(test_idx)[0]
            train = np.where(~test_idx)[0]

            yield train, test

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedShuffleSplit, self).split(X, y, groups)

In [22]:
additonal_with_content_features_train = np.hstack((train[additonal_with_content].values, train_handmade_features))

additonal_with_content_features_test = np.hstack((test[additonal_with_content].values, test_handmade_features))

test_loader = torch.utils.data.DataLoader(TextDataset(x_test_q, x_test_a, features_test, additonal_with_content_features_test, test.index),
                          batch_size=BATCH_SIZE, shuffle=False)

In [23]:
y = train.loc[:, y_columns].values

oof = np.zeros((len(train), 30))
test_pred = np.zeros((len(test), 30))

kf = MultilabelStratifiedKFold(n_splits=NFOLDS, random_state=SEED).split(train, y)

k = 0
for i, (train_idx, valid_idx) in enumerate(kf):
    print(f'fold {i+1}')

      
    gc.collect()
    
    train_loader = torch.utils.data.DataLoader(TextDataset(x_train_q, x_train_a, features_train, additonal_with_content_features_train,  train_idx, y),
                          batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(TextDataset(x_train_q, x_train_a, features_train, additonal_with_content_features_train, valid_idx, y),
                          batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)
    
    net = BertForSequenceClassification(additional=False)

    net.cuda()
    # net2.cuda()
    
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss_fn2 = torch.nn.BCEWithLogitsLoss()
    optimizer = AdamW(net.parameters(), lr=LR, eps=4e-5)
    for epoch in range(EPOCHS):  
        start_time = time.time()
        avg_loss = 0.0
        avg_loss_q = 0.0
        avg_loss_a = 0.0
        net.train()
        # net2.train()
        for data in train_loader:

            # get the inputs
            input_ids, input_masks, input_segments, input_ids_a, input_masks_a, input_segments_a, net_features, additional_content, labels = data
            pred_q, pred_a  = net(input_ids = input_ids.long().cuda(),
                             labels = None,
                             attention_mask = input_masks.cuda(),
                             token_type_ids = input_segments.cuda(),
                             input_ids_a = input_ids_a.long().cuda(),
                             attention_mask_a = input_masks_a.cuda(), 
                             token_type_ids_a = input_segments_a.cuda(),
                            additional_feature = net_features.cuda(),
                            )
            pred = torch.cat((pred_q, pred_a), 1)
            
            loss_q = loss_fn(pred_q, labels[:, :21].cuda())
            loss_a = loss_fn2(pred_a, labels[:, 21:].cuda())
            loss = loss_q + loss_a
            # Before the backward pass, use the optimizer object to zero all of the
            # gradients for the Tensors it will update (which are the learnable weights
            # of the model)
            

            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calling the step function on an Optimizer makes an update to its parameters
            optimizer.step()

            optimizer.zero_grad()
            
            avg_loss += loss.item()
            avg_loss_q += loss_q.item()
            avg_loss_a += loss_a.item()
        
        avg_val_loss = 0.0
        avg_val_loss_q = 0.0
        avg_val_loss_a = 0.0
        net.eval()

        valid_preds = np.zeros((len(valid_idx), 30))
        true_label = np.zeros((len(valid_idx), 30))
        for j, data in enumerate(val_loader):

            # get the inputs
            input_ids, input_masks, input_segments, input_ids_a, input_masks_a, input_segments_a, net_features, additional_content, labels = data

            ## forward + backward + optimize
            
            pred_q, pred_a  = net(input_ids = input_ids.long().cuda(),
                             labels = None,
                             attention_mask = input_masks.cuda(),
                             token_type_ids = input_segments.cuda(),
                             input_ids_a = input_ids_a.long().cuda(),
                             attention_mask_a = input_masks_a.cuda(), 
                             token_type_ids_a = input_segments_a.cuda(),
                            additional_feature = net_features.cuda(),
                            )
            pred = torch.cat((pred_q, pred_a), 1)
            loss_val_q = loss_fn(pred_q, labels[:, :21].cuda())
            loss_val_a = loss_fn2(pred_a, labels[:, 21:].cuda())
            loss_val = loss_val_q + loss_val_a
            avg_val_loss += loss_val.item()
            avg_val_loss_q += loss_val_q.item()
            avg_val_loss_a += loss_val_a.item()

            
            valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = torch.sigmoid(pred).cpu().detach().numpy()
            true_label[j * BATCH_SIZE:(j+1) * BATCH_SIZE]  = labels
        
        
        score = 0
        for i in range(30):
          s = np.nan_to_num(
                    spearmanr(true_label[:, i], valid_preds[:, i]).correlation / 30)
          score += s
        
        
        oof[valid_idx] = valid_preds
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f}  \t loss_q={:.4f} \t loss_a={:.4f} \t val_loss={:.4f} \t val_loss_q={:.4f} \t val_loss_a={:.4f} \t  spearman={:.4f} \t time={:.2f}s'.format(
            epoch + 1, EPOCHS, avg_loss / len(train_loader), avg_loss_q / len(train_loader), avg_loss_a / len(train_loader), \
             avg_val_loss / len(val_loader), avg_val_loss_q / len(val_loader),  avg_val_loss_a / len(val_loader), score, elapsed_time))
        
    
    k += 1
    torch.save(net.state_dict(), "bert_pytorch_base_folds_{}.pt".format(k))
    net.eval()
    test_pred_fold = np.zeros((len(test), 30))
    with torch.no_grad():
        for q, data in enumerate(test_loader):
            input_ids, input_masks, input_segments, input_ids_a, input_masks_a, input_segments_a, net_features, additional_content, labels = data

            pred_q, pred_a  = net(input_ids = input_ids.long().cuda(),
                             labels = None,
                             attention_mask = input_masks.cuda(),
                             token_type_ids = input_segments.cuda(),
                             input_ids_a = input_ids_a.long().cuda(),
                             attention_mask_a = input_masks_a.cuda(), 
                             token_type_ids_a = input_segments_a.cuda(),
                            additional_feature = net_features.cuda(),
                            )
            y_pred = torch.cat((pred_q, pred_a), 1)
            test_pred_fold[q * BATCH_SIZE:(q+1) * BATCH_SIZE] = torch.sigmoid(y_pred).cpu().detach().numpy()
        
    torch.cuda.empty_cache()
    gc.collect()
    test_pred += test_pred_fold/NFOLDS
        
        

fold 1
Epoch 1/2 	 loss=0.7822  	 loss_q=0.3979 	 loss_a=0.3843 	 val_loss=0.7365 	 val_loss_q=0.3700 	 val_loss_a=0.3664 	  spearman=0.3980 	 time=474.11s
Epoch 2/2 	 loss=0.7295  	 loss_q=0.3663 	 loss_a=0.3631 	 val_loss=0.7412 	 val_loss_q=0.3656 	 val_loss_a=0.3756 	  spearman=0.4000 	 time=472.44s
fold 2
Epoch 1/2 	 loss=0.7800  	 loss_q=0.3969 	 loss_a=0.3831 	 val_loss=0.7397 	 val_loss_q=0.3740 	 val_loss_a=0.3658 	  spearman=0.3837 	 time=474.71s
Epoch 2/2 	 loss=0.7280  	 loss_q=0.3661 	 loss_a=0.3619 	 val_loss=0.7401 	 val_loss_q=0.3688 	 val_loss_a=0.3713 	  spearman=0.3901 	 time=473.27s
fold 3
Epoch 1/2 	 loss=0.7831  	 loss_q=0.3998 	 loss_a=0.3833 	 val_loss=0.7409 	 val_loss_q=0.3674 	 val_loss_a=0.3735 	  spearman=0.3900 	 time=473.65s
Epoch 2/2 	 loss=0.7329  	 loss_q=0.3681 	 loss_a=0.3648 	 val_loss=0.7324 	 val_loss_q=0.3626 	 val_loss_a=0.3697 	  spearman=0.4036 	 time=473.69s
fold 4
Epoch 1/2 	 loss=0.7823  	 loss_q=0.3980 	 loss_a=0.3843 	 val_loss=0.7396 	 v

In [24]:
oof_score = 0
for i in range(30):
    oof_score += np.nan_to_num(
            spearmanr(y[:, i], oof[:, i]).correlation / 30)

In [25]:
oof_score

0.3885292260180297

In [26]:
sub.loc[:, y_columns] = test_pred
sub.to_csv('submission.csv', index=False)

In [27]:
sub.head(10)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.943629,0.655591,0.199252,0.527002,0.636663,0.596740,0.666586,0.634873,0.613811,...,0.917764,0.915923,0.626021,0.962519,0.962674,0.840686,0.036153,0.074343,0.796255,0.912743
1,46,0.859836,0.508686,0.006255,0.750617,0.800714,0.909791,0.556808,0.480095,0.127230,...,0.688408,0.939088,0.644043,0.965965,0.976975,0.864369,0.908648,0.141205,0.129611,0.923550
2,70,0.917395,0.685009,0.020419,0.722105,0.887749,0.932084,0.596328,0.461238,0.266321,...,0.881696,0.923190,0.600987,0.971232,0.965405,0.814571,0.080409,0.058143,0.905311,0.935257
3,132,0.894433,0.474203,0.006534,0.694094,0.773255,0.900602,0.556111,0.418047,0.126703,...,0.748515,0.955547,0.709927,0.968567,0.980789,0.893466,0.744199,0.173645,0.777755,0.914785
4,200,0.909479,0.482548,0.023460,0.809603,0.795816,0.878340,0.684032,0.582122,0.169670,...,0.676847,0.901841,0.658323,0.961381,0.960251,0.827610,0.280525,0.147515,0.473095,0.928682
5,245,0.948966,0.796303,0.042071,0.639509,0.932208,0.896765,0.629878,0.550286,0.338646,...,0.915386,0.953094,0.658297,0.977129,0.977204,0.882169,0.085993,0.099922,0.822954,0.939190
6,257,0.877492,0.507163,0.005759,0.727573,0.759690,0.903613,0.551119,0.410423,0.159648,...,0.739054,0.952830,0.697013,0.967430,0.979543,0.891300,0.786493,0.215282,0.645197,0.921336
7,267,0.947405,0.723740,0.165835,0.796666,0.812591,0.783927,0.655759,0.651067,0.204758,...,0.889887,0.902368,0.680611,0.962273,0.960051,0.816590,0.018601,0.038855,0.825287,0.947138
8,284,0.839610,0.394745,0.006398,0.686993,0.764310,0.921893,0.517954,0.428336,0.103370,...,0.695261,0.969559,0.678091,0.979271,0.986497,0.913185,0.629029,0.144782,0.727038,0.930241
9,292,0.942840,0.686946,0.018193,0.848419,0.845418,0.916063,0.676750,0.592957,0.147876,...,0.853036,0.917736,0.612644,0.960090,0.972590,0.826781,0.634859,0.098095,0.774560,0.921387


In [28]:
sub.describe()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.894018,0.594608,0.033322,0.714732,0.807849,0.859870,0.586125,0.481539,0.257501,...,0.796661,0.935318,0.668943,0.965038,0.971810,0.865850,0.532813,0.148211,0.525013,0.925178
std,2812.670060,0.044599,0.135410,0.060157,0.093191,0.087943,0.107971,0.050150,0.079771,0.195531,...,0.085808,0.029066,0.042677,0.012574,0.014417,0.042336,0.303631,0.053958,0.265537,0.023215
min,39.000000,0.757599,0.327065,0.003384,0.270911,0.464504,0.275191,0.493716,0.346994,0.023676,...,0.627161,0.717092,0.523565,0.871497,0.883014,0.579397,0.004074,0.013141,0.044694,0.733869
25%,2572.000000,0.864836,0.475272,0.006857,0.660560,0.770251,0.836549,0.543666,0.424493,0.103290,...,0.725241,0.923306,0.639318,0.960424,0.967553,0.847923,0.274871,0.110155,0.298145,0.915471
50%,5093.000000,0.896693,0.576770,0.010710,0.710736,0.812723,0.899376,0.578074,0.453199,0.198185,...,0.798645,0.941950,0.670256,0.967397,0.975853,0.875755,0.588666,0.153303,0.522841,0.930653
75%,7482.000000,0.929720,0.709907,0.026951,0.768440,0.859959,0.922874,0.621664,0.522915,0.378538,...,0.876928,0.954027,0.699697,0.973008,0.980805,0.894699,0.811294,0.185857,0.773146,0.940930
max,9640.000000,0.972550,0.882987,0.474059,0.953217,0.979043,0.964814,0.722616,0.748445,0.823458,...,0.949123,0.978208,0.775337,0.984264,0.988438,0.947534,0.953878,0.267514,0.976193,0.960213
